## Connect Google Drive and GPU



In [37]:
%reset

# connect google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# connect colab gpu
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Mounted at /content/drive
Sat Dec 18 23:04:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    32W / 250W |   1275MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------

## Import Needed Libraries, Paramaters and Functions

In [38]:
import sys
import time
import os.path
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils import prune
import torchvision
import matplotlib.pyplot as plt
from torch.utils.mobile_optimizer import optimize_for_mobile
from scipy import stats
from sklearn.utils import shuffle

SEED = 10
WINDOW_SIZE = 80
FEATURE_SIZE = 3
LABEL_SIZE = 6
BATCH_SIZE = 32
PATH = '/content/drive/MyDrive/CNNPaper'
RAW_DATA_PATH = PATH + '/data/WISDM/WISDM_ar_v1.1_raw.txt'

In [39]:
def read_data(file_path):
  """
    Read data from file_path
    Paramater:
      file_path: str
    Return:
      a DataFrame with the data and labels 
  """
  print("Start reading data ...")
  column_names = ['user', 'activity', 'timestamp', 'x-accel', 'y-accel', 'z-accel']
  data = pd.read_csv(file_path, header=None, names=column_names)
  print("Finish reading data ...")
  return data

def feature_normalize(data):
  """
    Normalize the feature data
    Paramater:
      data: a list of floats
    Return:
      a list of floats with normalized data
  """
  mu = np.mean(data, axis=0)
  sigma = np.std(data, axis=0)
  return (data - mu) / sigma

def dataset_normalize(dataset):
  """
    Normalize the whole dataset
    Paramater:
      dataset: a DataFrame with the data and labels 
    Return:
      a DataFrame with the normalized data and labels 
  """
  dataset.dropna(axis=0, how='any', inplace=True)
  print("Normalizing x-accel ...")
  dataset['x-accel'] = feature_normalize(dataset['x-accel'])
  print("Normalizing y-accel ...")
  dataset['y-accel'] = feature_normalize(dataset['y-accel'])
  print("Normalizing z-accel ...")
  dataset['z-accel'] = feature_normalize(dataset['z-accel'])
  return dataset

def windows(data, size):
  """
    Obatin the starting index and ending index according to window size
    Paramater:
      data: a list of floats
      size: int
    Return:
      Starting index, ending index
  """
  start = 0
  while start < data.count():
    yield int(start), int(start + size)
    start += (size / 2)
      
def dataset_segmentation(data):
  """
    Dataset segmentation according the window size
    Paramater:
      data: a list of floats
    Return:
      segments and labels 
  """
  print("Start segmentation with window size: ", WINDOW_SIZE)
  segments = np.empty((0, WINDOW_SIZE, FEATURE_SIZE))
  labels = np.empty((0))
  for (start, end) in windows(data['timestamp'], WINDOW_SIZE):
      x = data["x-accel"][start:end]
      y = data["y-accel"][start:end]
      z = data["z-accel"][start:end]
      if len(data["timestamp"][start:end]) == WINDOW_SIZE:
        segments = np.vstack([segments, np.dstack([x,y,z])])
        labels = np.append(labels, stats.mode(data["activity"][start:end])[0][0])
  labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)
  segments = segments.reshape(len(segments), FEATURE_SIZE, WINDOW_SIZE)
  print("Finish segmentation ...")
  return segments, labels

def train_valid_test_split(segments, classes, test_x, test_y, k_fold):
  """
    Split train, valid and test datase
    Paramater:
      segments: a list of input data
      classes: a list of classes data
      k: k fold cross validation
    Return:
      segments and labels 
  """
  print("Start dataset split... ")
  seg_len = len(segments)
  idx_val = [0, int(seg_len/5*1), int(seg_len/5*2), int(seg_len/5*3), int(seg_len/5*4), seg_len]
  train_range1 = range(0, idx_val[k_fold])
  valid_range = range(idx_val[k_fold], idx_val[k_fold+1])
  train_range2 = range(idx_val[k_fold+1], seg_len)

  train_x = np.concatenate((segments[train_range1], segments[train_range2]), axis=0)
  train_y = np.concatenate((classes[train_range1], classes[train_range2]), axis=0)
  valid_x = segments[valid_range]
  valid_y = classes[valid_range]

  # get train data
  train_data = []
  for i in range(len(train_x)):
    train_data.append([train_x[i], train_y[i]])
  
  # get valid data
  valid_data = []
  for i in range(len(valid_x)):
    valid_data.append([valid_x[i], valid_y[i]])
  
  # get test data
  test_data = []
  for i in range(len(test_x)):
    test_data.append([test_x[i], test_y[i]])
  print(len(train_data))
  print(len(valid_data))
  print(len(test_data))

  # generate DataLoader for each dataset
  trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
  validloader = torch.utils.data.DataLoader(valid_data, shuffle=True, batch_size=BATCH_SIZE)
  testloader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=BATCH_SIZE)
  
  print("Finish dataset split... ")
  return trainloader, validloader, testloader

## Load and Save Train, Test, Valid Dataset

In [ ]:
TRAIN_LOADER_PATH = PATH + '/model/train_loader'
VALID_LOAER_PATH = PATH + '/model/valid_loader'
TEST_LOADER_PATH = PATH + '/model/test_loader'

dataset = dataset_normalize(read_data(RAW_DATA_PATH))
segments, classes = dataset_segmentation(dataset)

In [ ]:
np.random.seed(SEED)
segments, classes = shuffle(segments, classes)
test_x = segments[range(int(len(segments)*0.8), len(segments))]
test_y = classes[range(int(len(classes)*0.8), len(classes))]
total_x = segments[range(0, int(len(segments)*0.8))]
total_y = classes[range(0, int(len(classes)*0.8))]
print(len(test_x))
print(len(test_y))
print(len(total_x))
print(len(total_y))

cross_valid_range = 5

for k in range(cross_valid_range):
  print("Start spliting for k = " + str(k))
  trainloader, validloader, testloader = train_valid_test_split(total_x, total_y, test_x, test_y, k)
  CROSS_TRAIN_LOADER_PATH = TRAIN_LOADER_PATH + str(k) + '.pkl'
  CROSS_VALID_LOADER_PATH = VALID_LOAER_PATH + str(k) + '.pkl'
  CROSS_TEST_LOADER_PATH = TEST_LOADER_PATH + str(k) + '.pkl'
  torch.save(trainloader, CROSS_TRAIN_LOADER_PATH)
  torch.save(validloader, CROSS_VALID_LOADER_PATH)
  torch.save(testloader, CROSS_TEST_LOADER_PATH)
  print("Finish data loading...")

## Load CNN Model and Other Helper Functions




In [40]:
NODE_SIZE = 128
KERNAL_SIZE = 10
LEARNING_RATE = 0.0001

k = 3

TRAIN_LOADER_PATH = PATH + '/model/train_loader'
VALID_LOAER_PATH = PATH + '/model/valid_loader'
TEST_LOADER_PATH = PATH + '/model/test_loader'
CROSS_TRAIN_LOADER_PATH = TRAIN_LOADER_PATH + str(k) + '.pkl'
CROSS_VALID_LOADER_PATH = VALID_LOAER_PATH + str(k) + '.pkl'
CROSS_TEST_LOADER_PATH = TEST_LOADER_PATH + str(k) + '.pkl'
trainloader = torch.load(CROSS_TRAIN_LOADER_PATH)
validloader = torch.load(CROSS_VALID_LOADER_PATH)
testloader = torch.load(CROSS_TEST_LOADER_PATH)

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    # Convolutional Layers
    self.features = nn.Sequential(
      nn.Conv1d(FEATURE_SIZE, NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False),
      nn.ReLU(),
      nn.Conv1d(NODE_SIZE, NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False),
      nn.ReLU(),
      nn.Conv1d(NODE_SIZE, NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False),
      nn.ReLU(),
      nn.Conv1d(NODE_SIZE, NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False),
      nn.ReLU(),
      nn.Conv1d(NODE_SIZE, NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False),
      nn.ReLU(),
    )
  
    self.fc1 = nn.Linear(NODE_SIZE*(WINDOW_SIZE-5*(KERNAL_SIZE-1)), 100)
    self.fc2 = nn.Linear(100, LABEL_SIZE)
    self.max = nn.Softmax(dim=1)

  def forward(self, x):
    x = self.features(x)
    x = x.view(x.shape[0], -1)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    x = self.max(x)
    return x

def train_save_CNN_model(TYPE, EPOCH_SIZE):
  # manually set random seed
  torch.backends.cudnn.deterministic = True
  torch.manual_seed(SEED)

  # set gpu device
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  net = CNN().double().to(device)

  # pick the criterion and optimizer
  criterion = nn.MultiLabelSoftMarginLoss()
  optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

  print("Learning rate %.5f, batch size %d, node size %d, kernal size %d" % (LEARNING_RATE, BATCH_SIZE, NODE_SIZE, KERNAL_SIZE))

  # initialization
  train_acc_list = []
  val_acc_list = []
  test_acc_list = []
  accuray = 0

  # start to train with epoches
  for epoch in range(EPOCH_SIZE):
    running_loss = 0.0
    train_total = 0
    train_correct = 0
    valid_total = 0
    valid_correct = 0
    test_total = 0
    test_correct = 0

    # for the training dataset
    for i, data in enumerate(trainloader, 0):
      inputs, labels = data
      inputs, labels = inputs.cuda(0), labels.cuda(0)
      optimizer.zero_grad()
      outputs = net(inputs)
      train_total += labels.size(0)
      train_correct += (torch.max(outputs, 1)[1] == torch.max(labels, 1)[1]).sum().item()
      loss = criterion(outputs, labels)
      if TYPE == 'l0_norm':
        # add group lasso regularization
        lgl = 1e-10
        cnt = torch.tensor([0]).cuda(0)
        for name, param in net.named_parameters():
          if "features" in name:
            cnt = cnt + param.detach().nonzero().size(0)
            #cnt = cnt + len(param.detach()[param.detach() > 1e-2]) + len(param.detach()[param.detach() < -1e-2])
        loss = loss + lgl * cnt
      elif TYPE == 'l1_norm':
        # add group lasso regularization
        lgl = 0.000001
        regularization = torch.tensor([0]).cuda(0)
        for name, param in net.named_parameters():
          if "features" in name:
            regularization = regularization + torch.norm(param, 1)
        loss = loss + lgl * regularization
      elif TYPE == 'l2_norm':
        lgl = 0.000001
        regularization = torch.tensor([0]).cuda(0)
        for name, param in net.named_parameters():
          if "features" in name:
            regularization = regularization + torch.norm(param)
        loss = loss + lgl * regularization
      elif TYPE == 'group_lasso':
        # add group lasso regularization
        lgl = 0.000001
        regularization = torch.tensor([0]).cuda(0)
        for name, param in net.named_parameters():
          if "features" in name:
            for i in range(param.shape[0]):
              regularization = regularization + torch.norm(param[i,:,:])
        loss = loss + lgl * regularization
      elif TYPE == 'l1_group_lasso':
        lgl = 0.000001
        alpha = 0.5
        group_lasso_regularization = torch.tensor([0]).cuda(0)
        lasso_regularization = torch.tensor([0]).cuda(0)
        for name, param in net.named_parameters():
          if "features" in name:
            for i in range(param.shape[0]):
              group_lasso_regularization = group_lasso_regularization + torch.norm(param[i,:,:])
            lasso_regularization = lasso_regularization + torch.norm(param, 1)
        loss = loss + (1-alpha) * lgl * group_lasso_regularization + alpha * lgl * lasso_regularization
      elif TYPE == 'l0_group_lasso':
        l0 = 1e-8
        lg = 0.4*1e-4
        cnt = torch.tensor([0]).cuda(0)
        group_lasso_regularization = torch.tensor([0]).cuda(0)
        lasso_regularization = torch.tensor([0]).cuda(0)
        for name, param in net.named_parameters():
          if "features" in name:
            for i in range(param.shape[0]):
              group_lasso_regularization = group_lasso_regularization + torch.norm(param[i,:,:])
            cnt += param.detach().nonzero().size(0)
        loss = loss + lg * group_lasso_regularization + l0 * cnt
      loss.backward()
      optimizer.step()
      running_loss += loss.item()

    # for the validation dataset
    for data in validloader:
      inputs, labels = data
      inputs, labels = inputs.cuda(0), labels.cuda(0)
      outputs = net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      valid_total += labels.size(0)
      valid_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    
    # for the test dataset
    for data in testloader:
      inputs, labels = data
      inputs, labels = inputs.cuda(0), labels.cuda(0)
      outputs = net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    
    # obtain the results for training, validation, test dataset
    train_acc = 100 * train_correct / train_total
    valid_acc = 100 * valid_correct / valid_total
    test_acc = 100 * test_correct / test_total
    train_acc_list.append(train_acc)
    val_acc_list.append(valid_acc)
    test_acc_list.append(test_acc)
    print("epoch %d, loss %.3f, train acc %.2f%%, valid acc %.2f%%, test acc %.2f%%" % (epoch+1, running_loss, train_acc, valid_acc, test_acc))
    
    # save the best model
    if valid_acc > accuray:
      accuray = valid_acc
      torch.save(net, PATH + '/model/' + TYPE + str(k) + ".ptl")
      torch.jit.save(torch.jit.script(net), PATH + '/model/' + TYPE + str(k) + "_git.ptl")
    
  return train_acc_list, val_acc_list, test_acc_list

## Results for CNN Model

In [ ]:
TYPE = 'no_penalty'
EPOCH_SIZE = 100
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)

Learning rate 0.00010, batch size 32, node size 128, kernal size 10
epoch 1, loss 375.590, train acc 66.53%, valid acc 78.90%, test acc 79.07%
epoch 2, loss 365.215, train acc 79.55%, valid acc 80.54%, test acc 80.66%
epoch 3, loss 364.266, train acc 80.43%, valid acc 80.13%, test acc 80.64%
epoch 4, loss 362.751, train acc 82.05%, valid acc 78.85%, test acc 78.66%
epoch 5, loss 361.505, train acc 83.37%, valid acc 84.91%, test acc 85.30%
epoch 6, loss 360.190, train acc 84.77%, valid acc 86.14%, test acc 86.23%
epoch 7, loss 359.274, train acc 85.87%, valid acc 86.23%, test acc 86.96%
epoch 8, loss 358.188, train acc 87.17%, valid acc 86.55%, test acc 87.16%
epoch 9, loss 357.482, train acc 87.83%, valid acc 86.16%, test acc 86.89%
epoch 10, loss 356.654, train acc 88.75%, valid acc 88.46%, test acc 88.87%
epoch 11, loss 355.866, train acc 89.61%, valid acc 88.73%, test acc 89.75%
epoch 12, loss 355.320, train acc 90.13%, valid acc 89.39%, test acc 89.60%
epoch 13, loss 354.394, train

In [ ]:
TYPE = 'l0_norm'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)

Learning rate 0.00010, batch size 32, node size 128, kernal size 10
epoch 1, loss 375.626, train acc 66.53%, valid acc 78.90%, test acc 79.07%
epoch 2, loss 365.251, train acc 79.55%, valid acc 80.54%, test acc 80.66%
epoch 3, loss 364.303, train acc 80.43%, valid acc 80.13%, test acc 80.64%
epoch 4, loss 362.787, train acc 82.05%, valid acc 78.85%, test acc 78.66%
epoch 5, loss 361.541, train acc 83.37%, valid acc 84.91%, test acc 85.30%
epoch 6, loss 360.226, train acc 84.77%, valid acc 86.14%, test acc 86.23%
epoch 7, loss 359.311, train acc 85.87%, valid acc 86.23%, test acc 86.96%
epoch 8, loss 358.225, train acc 87.17%, valid acc 86.55%, test acc 87.16%
epoch 9, loss 357.519, train acc 87.83%, valid acc 86.16%, test acc 86.89%
epoch 10, loss 356.691, train acc 88.75%, valid acc 88.46%, test acc 88.87%
epoch 11, loss 355.903, train acc 89.61%, valid acc 88.73%, test acc 89.75%
epoch 12, loss 355.357, train acc 90.13%, valid acc 89.39%, test acc 89.60%
epoch 13, loss 354.430, train

In [ ]:
TYPE = 'l1_norm'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)

Learning rate 0.00010, batch size 32, node size 128, kernal size 10
epoch 1, loss 380.463, train acc 65.42%, valid acc 77.24%, test acc 77.36%
epoch 2, loss 369.301, train acc 79.13%, valid acc 80.83%, test acc 80.70%
epoch 3, loss 367.502, train acc 81.10%, valid acc 81.77%, test acc 81.97%
epoch 4, loss 366.659, train acc 81.90%, valid acc 79.56%, test acc 79.22%
epoch 5, loss 366.131, train acc 82.36%, valid acc 81.90%, test acc 82.08%
epoch 6, loss 365.516, train acc 82.90%, valid acc 81.93%, test acc 81.99%
epoch 7, loss 365.016, train acc 83.35%, valid acc 83.88%, test acc 83.96%
epoch 8, loss 364.646, train acc 83.73%, valid acc 83.02%, test acc 83.50%
epoch 9, loss 364.158, train acc 84.18%, valid acc 81.83%, test acc 81.93%
epoch 10, loss 363.541, train acc 84.89%, valid acc 84.89%, test acc 85.27%
epoch 11, loss 362.757, train acc 85.36%, valid acc 85.52%, test acc 85.39%
epoch 12, loss 361.963, train acc 86.25%, valid acc 86.02%, test acc 86.34%
epoch 13, loss 360.848, train

In [ ]:
TYPE = 'l2_norm'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)

Learning rate 0.00010, batch size 32, node size 128, kernal size 10
epoch 1, loss 375.321, train acc 66.84%, valid acc 78.12%, test acc 78.73%
epoch 2, loss 365.143, train acc 79.64%, valid acc 80.88%, test acc 81.04%
epoch 3, loss 362.997, train acc 81.79%, valid acc 83.29%, test acc 83.23%
epoch 4, loss 361.872, train acc 83.02%, valid acc 83.31%, test acc 83.68%
epoch 5, loss 360.895, train acc 84.06%, valid acc 83.79%, test acc 84.37%
epoch 6, loss 359.976, train acc 85.09%, valid acc 84.45%, test acc 85.18%
epoch 7, loss 359.467, train acc 85.62%, valid acc 85.43%, test acc 86.60%
epoch 8, loss 358.318, train acc 86.95%, valid acc 86.02%, test acc 86.96%
epoch 9, loss 357.612, train acc 87.64%, valid acc 87.43%, test acc 88.40%
epoch 10, loss 356.852, train acc 88.63%, valid acc 87.73%, test acc 88.58%
epoch 11, loss 356.462, train acc 88.96%, valid acc 87.94%, test acc 88.36%
epoch 12, loss 355.372, train acc 90.28%, valid acc 88.60%, test acc 88.75%
epoch 13, loss 354.863, train

In [ ]:
TYPE = 'group_lasso'
EPOCH_SIZE = 100
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)

Learning rate 0.00010, batch size 32, node size 128, kernal size 10
epoch 1, loss 376.362, train acc 65.86%, valid acc 78.51%, test acc 78.47%
epoch 2, loss 365.537, train acc 79.37%, valid acc 79.56%, test acc 79.53%
epoch 3, loss 363.904, train acc 80.97%, valid acc 81.17%, test acc 81.04%
epoch 4, loss 362.496, train acc 82.50%, valid acc 83.11%, test acc 83.94%
epoch 5, loss 361.233, train acc 83.91%, valid acc 83.25%, test acc 84.21%
epoch 6, loss 360.563, train acc 84.62%, valid acc 84.57%, test acc 84.92%
epoch 7, loss 359.769, train acc 85.51%, valid acc 86.05%, test acc 86.56%
epoch 8, loss 359.074, train acc 86.44%, valid acc 85.93%, test acc 85.76%
epoch 9, loss 358.125, train acc 87.41%, valid acc 87.94%, test acc 87.20%
epoch 10, loss 357.957, train acc 87.65%, valid acc 85.45%, test acc 86.45%
epoch 11, loss 356.918, train acc 88.75%, valid acc 88.21%, test acc 88.84%
epoch 12, loss 356.462, train acc 89.22%, valid acc 85.11%, test acc 85.76%
epoch 13, loss 355.871, train

In [ ]:
TYPE = 'l1_group_lasso'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)

Learning rate 0.00010, batch size 32, node size 128, kernal size 10
epoch 1, loss 377.526, train acc 66.80%, valid acc 78.90%, test acc 79.51%
epoch 2, loss 367.475, train acc 79.31%, valid acc 79.92%, test acc 80.55%
epoch 3, loss 366.480, train acc 80.29%, valid acc 80.83%, test acc 81.15%
epoch 4, loss 365.884, train acc 80.95%, valid acc 79.33%, test acc 79.53%
epoch 5, loss 363.769, train acc 83.22%, valid acc 84.04%, test acc 84.39%
epoch 6, loss 362.578, train acc 84.53%, valid acc 86.05%, test acc 85.90%
epoch 7, loss 361.814, train acc 85.42%, valid acc 84.57%, test acc 84.87%
epoch 8, loss 360.747, train acc 86.55%, valid acc 85.93%, test acc 86.34%
epoch 9, loss 360.803, train acc 86.40%, valid acc 86.80%, test acc 87.42%
epoch 10, loss 359.574, train acc 87.85%, valid acc 87.32%, test acc 88.14%
epoch 11, loss 359.064, train acc 88.41%, valid acc 87.50%, test acc 88.20%
epoch 12, loss 358.194, train acc 89.39%, valid acc 85.93%, test acc 86.03%
epoch 13, loss 357.889, train

In [ ]:
TYPE = 'l0_group_lasso'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)
print(train_acc)
print(valid_acc)
print(test_acc)
file_name = 'data'+str(k)+'.txt'
with open(file_name, 'w') as f:
    for i in train_acc:
      f.write("%f " % i)
    f.write('\n')
    for i in valid_acc:
      f.write("%f " % i)
    f.write('\n')
    for i in test_acc:
      f.write("%f " % i)
    f.write('\n')


Learning rate 0.00010, batch size 32, node size 256, kernal size 10
epoch 1, loss 420.618, train acc 42.15%, valid acc 64.76%, test acc 65.00%
epoch 2, loss 396.407, train acc 66.30%, valid acc 69.63%, test acc 69.70%
epoch 3, loss 391.825, train acc 70.27%, valid acc 71.00%, test acc 71.57%
epoch 4, loss 390.335, train acc 71.34%, valid acc 70.54%, test acc 71.35%
epoch 5, loss 389.033, train acc 72.66%, valid acc 71.93%, test acc 72.21%
epoch 6, loss 388.259, train acc 73.47%, valid acc 74.73%, test acc 75.29%
epoch 7, loss 387.539, train acc 74.41%, valid acc 75.10%, test acc 75.60%
epoch 8, loss 386.291, train acc 76.04%, valid acc 75.83%, test acc 76.27%
epoch 9, loss 385.626, train acc 77.08%, valid acc 76.42%, test acc 76.85%
epoch 10, loss 385.092, train acc 78.15%, valid acc 78.47%, test acc 78.47%
epoch 11, loss 384.304, train acc 79.14%, valid acc 77.01%, test acc 76.87%
epoch 12, loss 383.210, train acc 80.67%, valid acc 81.95%, test acc 81.42%
epoch 13, loss 383.643, train

## Results after Pruning the above Models

In [41]:
PRUNE_THRESHOLD = 0.015

class ThresholdPruning(prune.BasePruningMethod):
    PRUNING_TYPE = "unstructured"

    def __init__(self, threshold):
        self.threshold = threshold

    def compute_mask(self, tensor, default_mask):
      return torch.abs(tensor) > self.threshold

PATHS = {'No penalty - 3 128 0.0001':            PATH + '/model/final/lr0.0001/no_penalty3.ptl',
         'l0 norm - 3 128 0.0001':               PATH + '/model/final/lr0.0001/l0_norm3.ptl',
         'l1 norm - 3 128 0.0001':               PATH + '/model/final/lr0.0001/l1_norm3.ptl',
         'l2 norm - 3 128 0.0001':               PATH + '/model/final/lr0.0001/l2_norm3.ptl',
         'group lasso - 3 128 0.0001':           PATH + '/model/final/lr0.0001/group_lasso3.ptl',
         'l1 group lasso - 3 128 0.0001':        PATH + '/model/final/lr0.0001/l1_group_lasso3.ptl',
         'l0 group lasso - 3 128 0.0001 (BEST)': PATH + '/model/final/lr0.0001/l0_group_lasso3.ptl',
         'l0 group lasso - 3 128 0.00005':       PATH + '/model/final/lr0.00005/l0_group_lasso3.ptl',
         'l0 group lasso - 3 128 0.00015':       PATH + '/model/final/lr0.00015/l0_group_lasso3.ptl',
         'l0 group lasso - 3 128 0.0002':        PATH + '/model/final/lr0.0002/l0_group_lasso3.ptl',
         'l0 group lasso - 3 128 0.00001':       PATH + '/model/final/lr0.00001/l0_group_lasso3.ptl',
         'l0 group lasso - 3 256 0.0001':        PATH + '/model/final/256node/l0_group_lasso3.ptl',
         'l0 group lasso - 3 64 0.0001':         PATH + '/model/final/64node/l0_group_lasso3.ptl',
         'l0 group lasso - 0 128 0.0001':        PATH + '/model/final/lr0.0001/l0_group_lasso0.ptl',
         'l0 group lasso - 1 128 0.0001':        PATH + '/model/final/lr0.0001/l0_group_lasso1.ptl',
         'l0 group lasso - 2 128 0.0001':        PATH + '/model/final/lr0.0001/l0_group_lasso2.ptl',
         'l0 group lasso - 4 128 0.0001':        PATH + '/model/final/lr0.0001/l0_group_lasso4.ptl',
        }

for name in PATHS:
  print('Here are the results for {}:'.format(name))
  # load the model
  net = torch.load(PATHS[name])

  # display the results before compressed model
  test_correct = 0
  test_total = 0
  with torch.no_grad():
    for data in testloader:
      inputs, labels = data
      inputs, labels = inputs.cuda(0), labels.cuda(0)
      outputs = net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    test_acc = 100 * test_correct / test_total
  print('Accuracy of the network on the %d test data: %.2f %% before compression' % (test_total, test_acc))

  # prune the model
  parameters_to_prune = []
  for name, child in net.features.named_children():
    if int(name) % 2 == 0:
      parameters_to_prune.append((child, "weight"))
  prune.global_unstructured(parameters_to_prune, pruning_method=ThresholdPruning, threshold=PRUNE_THRESHOLD)

  # calculate the sparsity
  total_weight = 0
  total_nonzero = 0
  for name, child in net.features.named_children():
    if int(name) % 2 == 0:
      total_weight += torch.numel(child.weight)
      total_nonzero += torch.count_nonzero(child.weight)
  print('Sparity for the compressed model: %.2f %%' % (100*float(total_nonzero / total_weight)))

  # display the results after compressed model
  test_correct = 0
  test_total = 0
  with torch.no_grad():
    for data in testloader:
      inputs, labels = data
      inputs, labels = inputs.cuda(0), labels.cuda(0)
      outputs = net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    test_acc = 100 * test_correct / test_total
  print('Accuracy of the network on the %d test data: %.2f %% after compression\n' % (test_total, test_acc))

Here are the results for No penalty - 3 128 0.0001:
Accuracy of the network on the 5491 test data: 94.79 % before compression
Sparity for the compressed model: 60.10 %
Accuracy of the network on the 5491 test data: 94.63 % after compression

Here are the results for l0 norm - 3 128 0.0001:
Accuracy of the network on the 5491 test data: 94.72 % before compression
Sparity for the compressed model: 63.36 %
Accuracy of the network on the 5491 test data: 94.79 % after compression

Here are the results for l1 norm - 3 128 0.0001:
Accuracy of the network on the 5491 test data: 94.30 % before compression
Sparity for the compressed model: 13.58 %
Accuracy of the network on the 5491 test data: 93.84 % after compression

Here are the results for l2 norm - 3 128 0.0001:
Accuracy of the network on the 5491 test data: 94.61 % before compression
Sparity for the compressed model: 56.28 %
Accuracy of the network on the 5491 test data: 94.54 % after compression

Here are the results for group lasso - 3 

# Threthold check

In [ ]:
PRUNE_THRESHOLD = np.arange(0.005, 0.03, 0.005)

class ThresholdPruning(prune.BasePruningMethod):
    PRUNING_TYPE = "unstructured"

    def __init__(self, threshold):
        self.threshold = threshold

    def compute_mask(self, tensor, default_mask):
      return torch.abs(tensor) > self.threshold

PATHS = PATH + '/model/final/lr0.0001/l0_group_lasso3.ptl'

for threshold in PRUNE_THRESHOLD:
  print('Here are the results for threshold {}:'.format(threshold))
  # load the model
  net = torch.load(PATHS)

  # display the results before compressed model
  test_correct = 0
  test_total = 0
  with torch.no_grad():
    for data in testloader:
      inputs, labels = data
      inputs, labels = inputs.cuda(0), labels.cuda(0)
      outputs = net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    test_acc = 100 * test_correct / test_total
  print('Accuracy of the network on the %d test data: %.2f %% before compression' % (test_total, test_acc))

  # prune the model
  parameters_to_prune = []
  for name, child in net.features.named_children():
    if int(name) % 2 == 0:
      parameters_to_prune.append((child, "weight"))
  prune.global_unstructured(parameters_to_prune, pruning_method=ThresholdPruning, threshold=threshold)

  # calculate the sparsity
  total_weight = 0
  total_nonzero = 0
  for name, child in net.features.named_children():
    if int(name) % 2 == 0:
      total_weight += torch.numel(child.weight)
      total_nonzero += torch.count_nonzero(child.weight)
  print('Sparity for the compressed model: %.2f %%' % (100*float(total_nonzero / total_weight)))

  # display the results after compressed model
  test_correct = 0
  test_total = 0
  with torch.no_grad():
    for data in testloader:
      inputs, labels = data
      inputs, labels = inputs.cuda(0), labels.cuda(0)
      outputs = net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    test_acc = 100 * test_correct / test_total
  print('Accuracy of the network on the %d test data: %.2f %% after compression\n' % (test_total, test_acc))

Here are the results for threshold 0.005:
Accuracy of the network on the 5491 test data: 94.97 % before compression
Sparity for the compressed model: 18.96 %
Accuracy of the network on the 5491 test data: 95.12 % after compression

Here are the results for threshold 0.01:
Accuracy of the network on the 5491 test data: 94.97 % before compression
Sparity for the compressed model: 13.54 %
Accuracy of the network on the 5491 test data: 94.92 % after compression

Here are the results for threshold 0.015:
Accuracy of the network on the 5491 test data: 94.97 % before compression
Sparity for the compressed model: 9.52 %
Accuracy of the network on the 5491 test data: 94.65 % after compression

Here are the results for threshold 0.02:
Accuracy of the network on the 5491 test data: 94.97 % before compression
Sparity for the compressed model: 6.59 %
Accuracy of the network on the 5491 test data: 94.01 % after compression

Here are the results for threshold 0.025:
Accuracy of the network on the 549